## Based on work from [Chechik et al., 1998.](https://www-mitpressjournals-org.proxy.lib.duke.edu/doi/pdf/10.1162/089976698300017124)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy

In [3]:
M = 50 #number of memories
N = 100 #number of neurons
timesteps = 2 #number of timesteps for the simulation


#creating patterns
patterns = np.empty(shape = (M, N))
for m in range(M):
    patterns[m] = np.random.choice([-1, 1], N)

In [36]:
#creating the initial connectivity matrix based on a sum of the patterns - THIS SEEMS TO BE THE BOTTLENECK
connectivity = np.zeros(shape = (N, N))
for m in range(M):
    connectivity += np.outer(patterns[m], patterns[m].T)
for i in range(N):
    connectivity[i][i] = 0

Wij = connectivity / np.sqrt(M)
z = Wij.copy()
t = .3

In [37]:
counts = np.unique(Wij, return_counts = True)
zeros = int(counts[1][counts[0] == 0])

zeros

1282

In [38]:
## random deletion

indices = np.random.choice(np.arange(z.size), replace=False, size=int(z.size * t))
z_flat = z.flatten()
z_flat[indices] = 0
z = z_flat.reshape((100, 100))

#set the connectivity matrix to the modified/deleted synaptic values
Wij = z.copy()

In [39]:
counts = np.unique(Wij, return_counts = True)
zeros = int(counts[1][counts[0] == 0])

zeros

3868

In [43]:
def hopfield_model_synaptic_deletion_random(mem, neu, tim, threshold):

    import numpy as np
    import copy
    
    M = mem #number of memories
    N = neu #number of neurons
    timesteps = tim #number of timesteps for the simulation


    #creating patterns
    patterns = np.empty(shape = (M, N))
    for m in range(M):
        patterns[m] = np.random.choice([-1, 1], N)


    #creating the initial connectivity matrix based on a sum of the patterns - THIS SEEMS TO BE THE BOTTLENECK
    connectivity = np.zeros(shape = (N, N))
    for m in range(M):
        connectivity += np.outer(patterns[m], patterns[m].T)
    for i in range(N):
        connectivity[i][i] = 0
    
    Wij = connectivity / np.sqrt(M)
    z = Wij.copy()
    t = threshold
    
    ## random deletion

    indices = np.random.choice(np.arange(z.size), replace=False, size=int(z.size * t))
    z_flat = z.flatten()
    z_flat[indices] = 0
    z = z_flat.reshape((Wij.shape))

    
    #set the connectivity matrix to the modified/deleted synaptic values
    Wij = z.copy()

    #want an initial overlap of 0.8
    X = np.zeros(shape = (timesteps, N))
    X[0] = patterns[0].copy()
    n_change = int(X[0].shape[0] * 0.1)
    X[0][1:n_change] = -X[0][1:n_change]
    
    
    #update the model for every timestep based on its sign
    for t in range(timesteps - 1):
        X[t+1,:] = np.sign(Wij.dot(X[t,:]))

    #create an array to hold the overlap percentages   
    m_overlap = np.empty(shape = (timesteps, M))
    mu = []

    #calculate overlap percentages
    for t in range(timesteps):
        for u in range(M):
            mu = []
            for j in range(N):
                mu = np.append(mu, patterns[u][j] * X[t][j])
            m_overlap[t][u] = ((1/N) * mu.sum())
            
    #get the number of zeros that resulted in the Wij connectivity matrix following this pruning step
    counts = np.unique(Wij, return_counts = True)       
    zeros = counts[1][counts[0] == 0]        
            
    return(m_overlap.take(0, axis = 1)[1], zeros)

In [61]:
%%time

del_levels = np.arange(0, 1.05, .05)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_random(mem = int(memory[i]), neu = 800, tim = 2, threshold = 0 + lvl)

Wall time: 13min 18s


In [62]:
random1 = memory.copy()
random_x1 = x.copy()

In [63]:
random1, random_x1

(array([125., 117.,  98.,  95.,  99.,  82.,  92.,  82.,  80.,  64.,  57.,
         53.,  47.,  43.,  37.,  30.,  25.,  19.,  15.,   7.,   1.]),
 array([[9.42500e-01, 8.00000e+02],
        [9.45000e-01, 3.27630e+04],
        [9.40000e-01, 1.10866e+05],
        [9.32500e-01, 9.66720e+04],
        [9.35000e-01, 1.28641e+05],
        [9.47500e-01, 2.02401e+05],
        [9.47500e-01, 2.29781e+05],
        [9.22500e-01, 2.60773e+05],
        [9.40000e-01, 2.90514e+05],
        [9.47500e-01, 3.23479e+05],
        [9.47500e-01, 3.20379e+05],
        [9.42500e-01, 3.52387e+05],
        [9.42500e-01, 3.84329e+05],
        [9.42500e-01, 4.16249e+05],
        [9.35000e-01, 4.48222e+05],
        [9.42500e-01, 5.03031e+05],
        [9.47500e-01, 5.12143e+05],
        [9.47500e-01, 5.44115e+05],
        [9.32500e-01, 5.76063e+05],
        [9.47500e-01, 6.08049e+05],
        [0.00000e+00, 6.40000e+05]]))

In [64]:
%%time

del_levels = np.arange(0, 1.05, .05)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_random(mem = int(memory[i]), neu = 800, tim = 2, threshold = 0 + lvl)

Wall time: 12min 42s


In [65]:
random2 = memory.copy()
random_x2 = x.copy()

In [66]:
%%time

del_levels = np.arange(0, 1.05, .05)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_random(mem = int(memory[i]), neu = 800, tim = 2, threshold = 0 + lvl)

Wall time: 13min 15s


In [67]:
random3 = memory.copy()
random_x3 = x.copy()

In [69]:
random_mean = []
random_x_mean = []

for i in range(len(random1)):
    random_mean = np.append(random_mean, np.mean([random1[i], random2[i], random3[i]]))
    
for i in range(len(random_x1)):
    random_x_mean = np.append(random_x_mean, np.mean([random_x1[i][1], random_x2[i][1], random_x3[i][1]]))

In [70]:
df = pd.DataFrame({'n_mem':random_mean, 'n_syn_deleted':random_x_mean})

In [71]:
names = np.repeat('random', len(random_mean))
df['del_type'] = names

In [72]:
df['del_level'] = 100*(df['n_syn_deleted']/df['n_syn_deleted'].max())

In [73]:
df

,n_mem,n_syn_deleted,del_type,del_level
0,115.000000,800.000000,random,0.125000
1,104.000000,49657.000000,random,7.758906
2,106.000000,80090.333333,random,12.514115
3,101.666667,124924.000000,random,19.519375
4,96.333333,156484.000000,random,24.450625
5,88.333333,187917.333333,random,29.362083
6,86.666667,231029.000000,random,36.098281
7,78.666667,236601.000000,random,36.968906
8,80.333333,279449.666667,random,43.664010
9,66.000000,300124.666667,random,46.894479


In [74]:
df.to_csv('C:\\Users\\Minecraft in 4K\\Dropbox\\spr_2020_classes\\quant_neurobio\\final_results_mean_tidy_random.csv')